In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import torch
import numpy as np
import random
import os
import time
import pdb
from models import create_Inet
import torch
import torchvision
import torchvision.transforms as transforms
from scipy import interpolate
from utils.image_folder import ImageFolder
from utils import utils
from utils.utils import *
# from utils.Visualizer import Visualizer
import matplotlib.pyplot as plt
from IPython.display import clear_output
from options import DefaultConfig
opt = DefaultConfig()
torch.cuda.set_device(opt.gpu_ids)
torch.manual_seed(opt.seed)
torch.cuda.manual_seed(opt.seed)
torch.cuda.manual_seed_all(opt.seed)
np.random.seed(opt.seed)
random.seed(opt.seed)
# torch.backends.cudnn.deterministic = True

Interaction Network: initialized


In [3]:
opt.dataroot_val = '/home/zhangy23/dataset/coco_nogray/subval/'
dataset_val = ImageFolder(opt.dataroot_val,
                       transform=transforms.Compose([
                                   transforms.Resize(opt.loadSize),
                                   transforms.RandomResizedCrop(opt.fineSize),
                                   transforms.ToTensor()]))
dataset_loader_val = torch.utils.data.DataLoader(dataset_val, batch_size=opt.batch_size_val, shuffle=False)
dataset_val_size = len(dataset_val)
print('#validation images = %d' % dataset_val_size)

#validation images = 1000


In [4]:
model = create_Inet(opt)
model.setup(opt)
path = './snapshot/I_auto_ep14_val_2_27.26.pkl'
model.load_state_dict(torch.load(path, map_location='cuda:2').state_dict())
model.eval();
model.cuda();

Interaction net was created


/home/zhangy23/anaconda3/envs/vc/lib/python3.6/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'models.interaction_net.Inet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [15]:
val_loss = 0
val_psnr = 0
for ii, data_raw_val in enumerate(dataset_loader_val):
    # auto colourisartion
    opt.no_prev = True
    #data_raw_val, pp = data_raw_val
    data_cpu_val = utils.get_colorization_data(data_raw_val, opt, p=opt.sample_p)
    if(data_cpu_val is None):
        continue
    data_val = ToCuda(data_cpu_val)
    fake_ab_val = model(data_val['gray'], data_val['clicks'], data_val['prev'])
    prev_ab_val = fake_ab_val.detach().cpu()

    # update first colourisation
    data_cpu_val = utils.get_colorization_data(data_raw_val, opt, prev=prev_ab_val, p=opt.sample_p, num_points = 10)
    # import pdb; pdb.set_trace()
    data_val = ToCuda(data_cpu_val)
    fake_ab_val = model(data_val['gray'], data_val['clicks'], data_val['prev'])
    loss_val = model.calc_loss(data_val['ab'], fake_ab_val)
    psnr_val = calc_batch_psnr(data_val['gray'].detach(), data_val['ab'].detach(), fake_ab_val.detach(), opt, avg=False)

    val_loss += loss_val.detach().cpu().numpy()
    val_psnr += psnr_val
    # break
print(val_psnr / dataset_val_size)

24.57296974182129


In [9]:
from sklearn.externals import joblib
joblib.dump(data_cpu_val, 'l2.pkl') 

['l2.pkl']

In [7]:
data_cpu_val['clicks']

tensor([[[[-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          ...,
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000]],

         [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  

In [11]:
print(torch.ones(14, 14))

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])


In [20]:
int(opt.fineSize/opt.mean_kernel)

14